In [1]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
import os

from dotenv import load_dotenv

load_dotenv()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

In [2]:
## embedding model
def get_embedding_model(model_name="/teamspace/studios/this_studio/bge-small-en-v1.5"):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    return embed_model

# generator model
def get_llm(model_name="llama3-8b-8192"):
    llm = Groq(model=model_name, api_key=os.getenv("GROQ_API"), temperature=0.8)
    return llm

## get deeplake vector database
def get_vector_database(id, dataset_name):
    my_activeloop_org_id = id # "hunter"
    my_activeloop_dataset_name = dataset_name # "Vietnamese-law-RAG"
    dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=False,
    )
    return vector_store

def get_index(vector_store):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    return index

In [3]:
## query generation / rewriting
from llama_index.core import PromptTemplate

query_str = "Vượt đèn đỏ sẽ bị gì?"

prompt_for_generating_query = PromptTemplate(
"""Bạn là một trợ lý xuất sắc trong việc tạo ra các câu truy vấn tìm kiếm liên quan. Dựa trên câu truy vấn đầu vào dưới đây, hãy tạo ra {num_queries} truy vấn tìm kiếm liên quan, mỗi câu trên một dòng. Lưu ý, trả lời bằng tiếng Việt và chỉ trả về các truy vấn đã tạo ra.

### Câu truy vấn đầu vào: {query}

### Các câu truy vấn:"""
)

def generate_queries(llm, query_str, num_queries=4):
    fmt_prompt = prompt_for_generating_query.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries =  response.text.split("\n")
    return queries

def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.retrieve(query))
    
    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, tasks)):
        results_dict[(query, i)] = query_result
    
    return results_dict

In [5]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

## set embedding model
embed_model = get_embedding_model()
Settings.embed_model = embed_model

## set llm 
llm = Ollama(model="vistral", request_timeout=120.0, max_new_tokens=2000) # get_llm()
Settings.llm = llm

## get vector store, index, and two retrievers
vector_store = get_vector_database("hunter", "Vietnamese-legal-data")
index = get_index(vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=3)

Deep Lake Dataset in hub://hunter/Vietnamese-legal-data already exists, loading from the storage


# HyDE

In [4]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [7]:
query_engine = index.as_query_engine()

In [8]:
query_str = "Vượt đèn đỏ bị phạt bao nhiêu tiền?"
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))